In [1]:
pip install --upgrade MetaTrader5

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo
import pytz
import time

In [4]:
# Initialize MetaTrader 5 connection
if not mt5.initialize():
    print("Initialization failed")
    quit()

DEFINING FUNCTIONS

In [5]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame_input):
    rates_frame_input['time'] = pd.to_datetime(rates_frame_input['time'], unit='s')
    rates_frame_input.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame_input["time"] = pd.to_datetime(rates_frame_input["time"])
    return rates_frame

def append_new_data(rates_frame, rates_frame_to_append):
    rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
    return rates_frame

def download_recent_OHLCdata():
    symbol = "EURUSD"
    timeframe = mt5.TIMEFRAME_M1
    
    #GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
    utc_plus_2 = timezone(timedelta(hours=2))
    utc_from = datetime.now(utc_plus_2) - timedelta(minutes=5) + timedelta(hours=2)
    utc_to = datetime.now(utc_plus_2) + timedelta(hours=2) - timedelta(minutes=1)
    startdate=utc_from.strftime('%Y%m%d')
    enddate=utc_to.strftime('%Y%m%d')
    
    # Get the OHLC data FOR TODAY AND YESTERDAY
    rates_5m = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    # Check if the data is downloaded successfully
    if rates is None:
        print("No data available, error code =", mt5.last_error())
    else:
        # Convert the data to a pandas DataFrame
        rates_frame_to_append = pd.DataFrame(rates_5m)
        df_convert_hr(rates_frame_to_append)
        return rates_frame_to_append

In [10]:
#Detect the entry drills
def check_buy_entry_drill():
    #check if pattern occured during london session:
    ldn_session = 10 < LowestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during ny session
    ny_session = 15 < LowestPoint["time"].to_pydatetime().hour < 19
    #check if the drill appreared during pre-session times
    pre_ldn_session = 9 < LowestPoint["time"].to_pydatetime().hour <= 10
    pre_ny_session = 14 <= LowestPoint["time"].to_pydatetime().hour <= 15

    if ldn_session and (ldn_low == LowestPoint["low"]):
        print(f"London session: {ldn_session}")
        print(f"London session low: {ldn_low}")
        print(f"Lowest point: {LowestPoint["low"]}")
        pass
    elif ny_session and (ny_low == LowestPoint["low"]):
        print(f"NY session: {ny_session}")
        print(f"NY session low: {ny_low}")
        print(f"Lowest point: {LowestPoint["low"]}")
        pass
    elif (pre_ldn_session or pre_ny_session):
        print(f"Pre session: {pre_ldn_session or pre_ny_session}")
        print(f"Lowest point: {LowestPoint["low"]}")
        pass
    else:
        time.sleep(60)
        rates_frame_to_append = download_recent_OHLCdata()
        rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
        print(rates_frame.tail(10))

def check_sell_entry_drill():
    #check if pattern occured during london session:
    ldn_session = 10 < HighestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during ny session
    ny_session = 15 < HighestPoint["time"].to_pydatetime().hour < 19
    #check if the drill appreared during pre-session times
    pre_ldn_session = 9 < HighestPoint["time"].to_pydatetime().hour <= 10
    pre_ny_session = 14 <= HighestPoint["time"].to_pydatetime().hour <= 15
    
    if ldn_session and (ldn_high == HighestPoint["high"]):
        print(f"London session: {ldn_session}")
        print(f"London session high: {ldn_high}")
        print(f"Highest point: {HighestPoint["high"]}")
        pass
    elif ny_session and (ny_high == HighestPoint["high"]):
        print(f"NY session: {ny_session}")
        print(f"NY session high: {ny_high}")
        print(f"Highest point: {HighestPoint["high"]}")
        pass
    elif (pre_ldn_session or pre_ny_session):
        print(f"Pre-session: {pre_ldn_session or pre_ny_session}")
        print(f"Highest point: {HighestPoint["high"]}")
        pass
    else:
        time.sleep(60)
        rates_frame_to_append = download_recent_OHLCdata()
        rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
        print(rates_frame.tail(10))
        

DOWNLOAD THE OHLC DATA FROM THE LAST 7 DAYS

In [7]:
symbol = "EURUSD"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
utc_from = datetime.now() - timedelta(days=1) + timedelta(hours=2)
utc_to = datetime.now() + timedelta(hours=2) - timedelta(minutes=1)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data FOR TODAY AND YESTERDAY
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    df_convert_hr(rates_frame)


CREATING A RISK MANAGEMENT STRATEGY THAT TRADES ONLY A MAXIMUM OF 4 TRADES PER DAY

In [9]:
#number of trades for today
N = 0
date_today = date.today()

while N < 4:
    if N > 4:
        break
    else:
        #HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
        ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 10) & (rates_frame['time'].dt.hour < 14)]
        ldn_high = ldn_rates_frame["high"].max()
        ldn_low = ldn_rates_frame["low"].min()
        print("__________________________________")
        print(f'London high: {ldn_high}')
        print(f'London low: {ldn_low}')
        
        #HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
        ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 15) & (rates_frame['time'].dt.hour < 19)]
        ny_high = ny_rates_frame["high"].max()
        ny_low = ny_rates_frame["low"].min()
        print(f'NY high: {ny_high}')
        print(f'NY low: {ny_low}')
        print("__________________________________")
        
        #Determine the max and min value of the highs and lows for the last 35 mins
        Last_35 = rates_frame.tail(35)
        MaxHighPrice = Last_35["high"].max()
        MinLowPrice = Last_35["low"].min()

        #Determine the if the last 35 mins is in a downtrend or uptrend
        HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
        LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]

        #Based on the trend (and the conditions inside) decide whether to buy, sell, or wait.
        if HighestPoint["time"] > LowestPoint["time"]:
            trend_35 = "uptrend"
            print(f"Trend: {trend_35}")
            check_sell_entry_drill()
            continue
        elif HighestPoint["time"] < LowestPoint["time"]:
            trend_35 = "downtrend"
            print(f"Trend: {trend_35}")
            check_buy_entry_drill()
        else:
            trend_35 = "sideways"
            print(f"Trend: {trend_35}")
            time.sleep(60)

__________________________________
London high: 1.04343
London low: 1.04031
NY high: nan
NY low: nan
__________________________________
Trend: downtrend
Pre session: True
Lowest point: 1.03871
__________________________________
London high: 1.04343
London low: 1.04031
NY high: nan
NY low: nan
__________________________________
Trend: downtrend
Pre session: True
Lowest point: 1.03871
__________________________________
London high: 1.04343
London low: 1.04031
NY high: nan
NY low: nan
__________________________________
Trend: downtrend
Pre session: True
Lowest point: 1.03871
__________________________________
London high: 1.04343
London low: 1.04031
NY high: nan
NY low: nan
__________________________________
Trend: downtrend
Pre session: True
Lowest point: 1.03871
__________________________________
London high: 1.04343
London low: 1.04031
NY high: nan
NY low: nan
__________________________________
Trend: downtrend
Pre session: True
Lowest point: 1.03871
__________________________________



KeyboardInterrupt

